# Probabilist PCA and Factor Analysis

## Probabilistic PCA

Probabilistic PCA is a probabilistic generative model for PCA that clarifies the assumptions under which PCA performs well and allows integration with other probabilistic models, i.e., mixtures. It is a latent variable model that assumes observed data $x$ are generated through a linear process by unobserved latent variables $z$. The $x_n$ are D-dimensional vectors, so if the $z_n$ are M-dimensional, with $M < D$, we recover the dimensionality reduction capabilities of PCA.

Consider the following distribution for latent variables and the conditional distribution for the observed data:
$$
p(\vec{z}) = \mathcal{N}\left(  \vec{z} \mid \vec{0}, I \right)\\
p(\vec{x}\mid \vec{z}) = \mathcal{N}\left(  \vec{x} \mid W\vec{z} + \mu, \sigma^2 I \right).
$$
Meaning, the latent variables are uncorrelated normally distributed random variables with zero mean and unit variance, and the conditional density specifies that, given $\vec{z}$, the $\vec{x}$ are also uncorrelated gaussian random variables, with mean given by a linear transformation of the $\vec{z}$ and variance $\sigma^2$. This is an example of **gaussian linear model** that have the advantage of being exactly solvable for the parameters $W$ and $\sigma$. Note $W$ is a $D\times M$ matrix.

It is often useful to take a generative point of view when discussing latent variable models. In this case, a random observation $\vec{x}$ is generated from a sampled value $\vec{z}$ through the equation:
$$
\vec{x} = W\vec{z} + \vec{\mu} + \vec{\epsilon},
$$
where $\vec{\epsilon}$ is a vector of gaussian white noise, i.e., $E[\epsilon_i] = 0$ and $Cov[\epsilon_i\epsilon_j] = \sigma^2\delta_{ij}$.

It is useful to spend some time understanding this linear relation, so will build an example that maps 2-dimensional $z$ to 3-dimensional $x$. The matrix $W$ will span our feature sub-space. Each column of W specifies a direction in D space, but, being M-dimensional, allows only to build a M dimensional manifold embedded in the data space. This is analogous to the unit vectors of the transformation matrix U from PCA, with the difference that so far, W is not required to be an orthonormal matrix.

In [65]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib widget

# Prepare visualization
fig = plt.figure(figsize=(16 , 8))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122, projection='3d')

# Choose a mean vector mu
mu = np.array([0.5, 1, -1])

DP = 10
ax2.set_xlim(mu[0] - DP, mu[0] + DP)
ax2.set_ylim(mu[1] - DP, mu[1] + DP)
ax2.set_zlim(mu[2] - DP, mu[2] + DP)

# Choose to durections in D space.
# This vectors will compose the W matrix.
# Note w1 and w2 are not unit length nor orthogonal.
w1 = np.array([10, -2, 1])
w2 = np.array([2, 2, 1])
W = np.stack([w1, w2], axis=-1)

# plot direction vectors
ax2.plot([mu[0], mu[0]+ w1[0]], [mu[1], mu[1] + w1[1]], [mu[2],mu[2] + w1[2]], 'k-')
ax2.plot([mu[0], mu[0]+w2[0]], [mu[1], mu[1]+w2[1]], [mu[2], mu[2]+w2[2]], 'k-')

# Sample and transform
sigma = 1
z = np.random.normal(0, 1, size=(2,1000))
x = W @ z + mu[:,None]
eps = np.random.normal(0, sigma, size=x.shape)
x += eps

ax1.scatter(z[0,:], z[1,:])
ax2.scatter(x[0,:], x[1,:], x[2,:], alpha=0.3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

The idea of the visualization is to explore how the z variables get mapped into the x space, where the M most relevant correlations are encoded into the W matrix. To find the values of W and $\sigma$ a common approach is to use Maximum Likelihood. PPCA has the advantage the ML solution is exact, and it involves optimizing with respect to the marginal distribution:
$$
p(\vec{x}) = \int p(\vec{x}\mid\vec{z})p(\vec{z})d\vec{z} = \mathcal{N}(\vec{x}\mid\vec{mu},C)
$$

Since both the prior $p(\vec{z})$ and the conditional $p(\vec{x}\mid\vec{z})$ are Gaussian, it must be that the marginal $p(\vec{x})$ must also be Gaussian, and instead of evaluating the integral we can find the expected value and covariance from the linear relationship $\vec{x} = W\vec{z} + \mu + \epsilon$ (derived in class), remembering that since we are considering the marginal $z$ is no longer a constant. Doing so gives $E[\vec{x}] = \mu$ and the covariance matrix is
$$
C = WW^T + \sigma^2 I
$$

The likelihood function is then that of a product of multivariate Gaussians (under the assumption of independent $x_n$), and the log-lokelihood is
$$
\begin{align}
\ln\mathcal{L} =& \ln p(X\mid\vec{mu},W,\sigma^2)
= \sum_{i=1}^N \ln p(\vec{x}\mid W,\mu,\sigma^2)\\
=& -\frac{ND}{2}\ln(2\pi) - \frac{N}{2}\ln\lvert C\rvert 
-\frac{1}{2}\sum_{n=1}^N \left( \vec{x}_n - \vec{\mu} \right)^TC^{-1}\left(\vec{x}_n -\vec\mu \right)
\end{align}
$$

Standard optimization leads to the following solutions:
$$
\begin{align}
\frac{\partial \ln\mathcal{L}}{\partial \vec{\mu}} = 0 \rightarrow& \vec{\mu}_{ML} = \bar{x}\\
\frac{\partial \ln\mathcal{L}}{\partial \vec{W}} = 0 \rightarrow& W_{ML} = U_M(L_M-\sigma^2 I)^{1/2}R\\
\frac{\partial \ln\mathcal{L}}{\partial \vec{\sigma^2}} = 0 \rightarrow& \sigma^2_{ML} = \frac{1}{D-M}\sum_{i=M+1}^D\lambda_i,
\end{align}
$$
where $U_M$ is the matrix with the first M eigenvectors of the covariance matrix S, $L_M$ is the diagonal matrix with the first M largest eigenvalues $\lambda_i$, and $R$ is an arbitrary orthogonal matrix representing rotations in feature space. If $R=I$, then $W$ is orthogonal, otherwise it is not. Let's interpret the above solutions.


The mean is the standard ML solution for a multivariate Gaussian model. The variance $\sigma^2$ is just the mean value if the variance in all discarded directions along the unused eigenvectors of S, meaning that the unexplained variance, orthogonal to the subspace is just approximated by its mean. The term $L_m-\sigma^2 I$ is a weight matrix for the the eigenvector matrix $U$.

Assuming $R=I$, we can interpret the weights $L_m-\sigma^2 I$ of the eigenvectors $\hat{u}_i$. In the above solution, each eigenvector is weighted by $\sqrt{\lambda_i - \sigma^2}$. This weights come from the fact the posteriori covariance is given by $C = WW^T + \sigma^2 I$. This means that each column vector of $W$ has a variance given by $\lambda_i - \sigma^2$ plus an isotropic variance in all directions given by $\sigma^2$, giving the expected sum $\lambda_i$ predicted by standard PCA. Explicit calculations of the variances in a direction orthogonal to the subspace $\hat v$ ($\hat{v}^T\hat{u_i}=0$) gives $v^T C v = \sigma^2$ and along a principal component $u_i^T C u_i = \lambda_i$ as expected.

To perform dimensionality reduction we need a way to obtain the $\vec{z_n}$ corresponding to each $\vec{x}_n$. A natural way to do this is to use $\vec{z_n} = E[\vec{z}\mid\vec{x}]$ for which we need the posteriori density $p(\vec{z}\mid\vec{x})$, which can be found by applying Bayes theorem and expanding the exponent of the resulting Gaussian. The general result for linear Gaussian models can be found in Bishop Chapter 2.
$$
p(\vec{z}\mid\vec{x}) = \frac{p(\vec{x}\mid\vec{z})p(\vec{z})}{p(\vec{x})} = \mathcal{N}\left( M^{-1}W^T(\vec{x}-\vec{\mu}), \sigma^2 M^{-1}  \right).
$$
From which is easy to verify
$$
\vec{z_n} = E[\vec{z}\mid\vec{x}] = M^{-1}W^T_{ML}(\vec{x}_n-\bar{x}),
$$
where $M = W^T W +\sigma^2 I$. Our calculation for $z_n$ in turns maps again into data space into $W E[z\mid x] + \mu$, which is the expected value of $x_n$, without the noise.

A straightforward way to compute PPCA is to obtain the eigenvectors and eigenvalues of $S$ and calculate $W$ and $\sigma$ as above. But this will not take full advantage of the PPCA model. The fact that PPCA have a probabilistic model means we can exploit approaches such as the Expectation Maximization algorithm to deal with missing data in a Bayesian way. Another advantage is that we now posses a generative model of the data that we can use to simulate new observations. We can also use the model in a mixture model to better represent complex multimodal data. Finally, its easy to remove the isotropic restrictions on the covariance matrix, which leads to the formulation of Factor Analysis, which will be discussed next. The exact same projections as PCA can be recovered in the limit $\sigma^2 \rightarrow 0$.

### The EM algorithm, an introduction and its application to PPCA.